# SPR 2026 - Exploratory Data Analysis

Análise exploratória dos dados de mamografia para classificação BI-RADS.

**Compatível com:** Google Colab / Kaggle Notebooks

In [ ]:
# ============================================================
# SETUP - Detecção de ambiente e paths
# ============================================================
import os
import sys

IS_KAGGLE = os.path.exists('/kaggle/input')
IS_COLAB = 'google.colab' in sys.modules

print(f"Ambiente: {'Kaggle' if IS_KAGGLE else 'Colab' if IS_COLAB else 'Local'}")

if IS_KAGGLE:
    DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
    OUTPUT_DIR = '/kaggle/working'
elif IS_COLAB:
    !pip install kaggle -q
    from google.colab import userdata
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    !mkdir -p ~/.kaggle
    !echo '{"username":"'$KAGGLE_USERNAME'","key":"'$KAGGLE_KEY'"}' > ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle competitions download -c spr-2026-mammography-report-classification -q
    !mkdir -p data && unzip -o -q spr-2026-mammography-report-classification.zip -d data/
    DATA_DIR = 'data'
    OUTPUT_DIR = '.'
else:
    DATA_DIR = '../data'
    OUTPUT_DIR = '.'

print(f"DATA_DIR: {DATA_DIR}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_colwidth', 200)

## 1. Carregar Dados

In [ ]:
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
print(f"Train shape: {train.shape}")
print(f"Colunas: {train.columns.tolist()}")

# Test (Code Competition - pode não existir em dev)
test_path = os.path.join(DATA_DIR, 'test.csv')
if os.path.exists(test_path):
    test = pd.read_csv(test_path)
    print(f"Test shape: {test.shape}")
else:
    test = None
    print("test.csv não disponível - existe apenas no runtime de avaliação Kaggle")

In [ ]:
train.info()
train.head()

## 2. Distribuição do Target (BI-RADS)

In [ ]:
BIRADS_LABELS = {
    0: 'Incompleto', 1: 'Negativo', 2: 'Benigno', 3: 'Provavelmente Benigno',
    4: 'Suspeito', 5: 'Altamente Sugestivo', 6: 'Malignidade Comprovada'
}

target_counts = train['target'].value_counts().sort_index()
print("Distribuição do Target:")
for idx, count in target_counts.items():
    print(f"  {idx} ({BIRADS_LABELS[idx]}): {count} ({count/len(train)*100:.1f}%)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = sns.color_palette('husl', 7)

axes[0].bar(target_counts.index, target_counts.values, color=colors)
axes[0].set_xlabel('BI-RADS Category')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribuição das Classes BI-RADS')

axes[1].pie(target_counts.values, labels=[f'{i}' for i in target_counts.index], 
            autopct='%1.1f%%', colors=colors)
axes[1].set_title('Proporção das Classes')

plt.tight_layout()
plt.show()

## 3. Análise dos Textos

In [ ]:
train['text_length'] = train['report'].apply(len)
train['word_count'] = train['report'].apply(lambda x: len(x.split()))

print("Estatísticas de Comprimento:")
print(train[['text_length', 'word_count']].describe())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
train.boxplot(column='text_length', by='target', ax=axes[0])
axes[0].set_title('Caracteres por Classe')
train.boxplot(column='word_count', by='target', ax=axes[1])
axes[1].set_title('Palavras por Classe')
plt.suptitle('')
plt.tight_layout()
plt.show()

In [ ]:
print("Exemplos de relatórios por classe:")
for target in sorted(train['target'].unique()):
    sample = train[train['target'] == target].iloc[0]
    print(f"\nBI-RADS {target} ({BIRADS_LABELS[target]}):")
    print("-"*40)
    print(sample['report'][:400] + "..." if len(sample['report']) > 400 else sample['report'])

In [ ]:
print("="*60)
print("RESUMO")
print("="*60)
print(f"Amostras: {len(train)} | Classes: {train['target'].nunique()}")
print(f"Palavras média: {train['word_count'].mean():.0f} | Max chars: {train['text_length'].max()}")
print(f"Imbalance ratio: {target_counts.max() / target_counts.min():.1f}x")

# SPR 2026 Mammography Report Classification
## Exploratory Data Analysis (EDA)

Este notebook realiza uma análise exploratória dos dados do desafio SPR 2026.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# Configurações
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Carregamento dos Dados

In [ ]:
# Carregar dados de treino
train_df = pd.read_csv('../data/train.csv')
print(f"Train shape: {train_df.shape}")
train_df.head()

In [ ]:
# Informações básicas
train_df.info()

## 2. Distribuição das Classes BI-RADS

In [ ]:
# Descrição das categorias BI-RADS
BIRADS_DESCRIPTIONS = {
    0: "Incompleto - necessita avaliação adicional",
    1: "Negativo - achado normal",
    2: "Achado benigno",
    3: "Provavelmente benigno",
    4: "Anormalidade suspeita",
    5: "Altamente sugestivo de malignidade",
    6: "Malignidade comprovada por biópsia",
}

# Distribuição
target_counts = train_df['target'].value_counts().sort_index()
print("Distribuição das categorias BI-RADS:")
for idx, count in target_counts.items():
    pct = count / len(train_df) * 100
    print(f"  {idx} - {BIRADS_DESCRIPTIONS[idx]}: {count} ({pct:.1f}%)")

In [ ]:
# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
colors = sns.color_palette('husl', n_colors=7)
ax1 = axes[0]
bars = ax1.bar(target_counts.index, target_counts.values, color=colors)
ax1.set_xlabel('Categoria BI-RADS', fontsize=12)
ax1.set_ylabel('Quantidade', fontsize=12)
ax1.set_title('Distribuição das Categorias BI-RADS', fontsize=14)
ax1.set_xticks(range(7))

# Adicionar valores nas barras
for bar, count in zip(bars, target_counts.values):
    ax1.annotate(f'{count}', 
                xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=10)

# Pie chart
ax2 = axes[1]
ax2.pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%', colors=colors)
ax2.set_title('Proporção das Categorias BI-RADS', fontsize=14)

plt.tight_layout()
plt.show()

## 3. Análise do Texto

In [ ]:
# Adicionar colunas de análise de texto
train_df['text_length'] = train_df['report'].str.len()
train_df['word_count'] = train_df['report'].str.split().str.len()

# Estatísticas
print("Estatísticas do texto:")
print(f"  Comprimento médio (chars): {train_df['text_length'].mean():.0f}")
print(f"  Comprimento mediano (chars): {train_df['text_length'].median():.0f}")
print(f"  Palavras médias: {train_df['word_count'].mean():.0f}")
print(f"  Min palavras: {train_df['word_count'].min()}")
print(f"  Max palavras: {train_df['word_count'].max()}")

In [ ]:
# Distribuição do comprimento do texto por classe
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot - comprimento
sns.boxplot(x='target', y='text_length', data=train_df, ax=axes[0], palette='husl')
axes[0].set_xlabel('Categoria BI-RADS', fontsize=12)
axes[0].set_ylabel('Comprimento do Texto (chars)', fontsize=12)
axes[0].set_title('Comprimento do Texto por Categoria', fontsize=14)

# Box plot - palavras
sns.boxplot(x='target', y='word_count', data=train_df, ax=axes[1], palette='husl')
axes[1].set_xlabel('Categoria BI-RADS', fontsize=12)
axes[1].set_ylabel('Número de Palavras', fontsize=12)
axes[1].set_title('Número de Palavras por Categoria', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# Histograma do comprimento
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(train_df['text_length'], bins=50, edgecolor='black', alpha=0.7)
ax.axvline(train_df['text_length'].median(), color='red', linestyle='--', label=f'Mediana: {train_df["text_length"].median():.0f}')
ax.set_xlabel('Comprimento do Texto', fontsize=12)
ax.set_ylabel('Frequência', fontsize=12)
ax.set_title('Distribuição do Comprimento do Texto', fontsize=14)
ax.legend()
plt.show()

## 4. Exemplos de Texto por Categoria

In [ ]:
# Exemplo de cada categoria
for category in range(7):
    sample = train_df[train_df['target'] == category].sample(1, random_state=42)
    print(f"\n{'='*60}")
    print(f"BI-RADS {category}: {BIRADS_DESCRIPTIONS[category]}")
    print(f"{'='*60}")
    print(sample['report'].values[0][:500] + "..." if len(sample['report'].values[0]) > 500 else sample['report'].values[0])

## 5. Word Clouds por Categoria

In [ ]:
# Word clouds para algumas categorias
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, category in enumerate([1, 2, 3, 4, 5, 6]):
    texts = ' '.join(train_df[train_df['target'] == category]['report'].tolist())
    wordcloud = WordCloud(width=400, height=300, background_color='white').generate(texts)
    
    axes[i].imshow(wordcloud, interpolation='bilinear')
    axes[i].axis('off')
    axes[i].set_title(f'BI-RADS {category}', fontsize=14)

plt.tight_layout()
plt.show()

## 6. Recomendações para Modelagem

Com base na análise:

1. **Desbalanceamento de Classes**: Considerar técnicas como:
   - Class weights
   - Oversampling/Undersampling
   - Focal Loss

2. **Comprimento do Texto**: A maioria dos textos é relativamente curta, então:
   - `max_length=512` deve ser suficiente
   - Considerar `max_length=256` para economia de memória

3. **Modelos Recomendados**:
   - **BERTimbau**: BERT treinado em português
   - **DeBERTa**: Excelente para classificação de texto
   - **T5**: Abordagem seq2seq
   - **GEMMA**: LLM mais recente com LoRA

4. **Estratégias**:
   - Validação cruzada estratificada (5-fold)
   - Ensemble de modelos